In [1]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from openai import OpenAI
from dotenv import load_dotenv
import os

In [2]:
load_dotenv()
openai = OpenAI(
    api_key= os.getenv('kurtim_api_key'), # Refer to Create a secret key section
    base_url="https://cloud.olakrutrim.com/v1",
)

In [3]:
vectors = FAISS.load_local("../VectorStores/nomic/AllPrinters", OllamaEmbeddings(model =  "nomic-embed-text"), allow_dangerous_deserialization= True)
retriever = vectors.as_retriever()

In [28]:
def multiQueryRetrieval(question):

    systemMessage = """You are an AI language model assistant. Your task is to generate five 
    different versions of the given user question to retrieve relevant documents from a vector 
    database. By generating multiple perspectives on the user question, your goal is to help
    the user overcome some of the limitations of the distance-based similarity search. 
    Provide these alternative questions separated by newlines. Do not add any further lines in the response except for the rephrased questions"""

    chat_completion = openai.chat.completions.create(
    model="Meta-Llama-3-8B-Instruct",
    messages=[
        {"role": "system", "content": systemMessage},
        {"role": "user", "content": question}
    ],
    logit_bias= {2435: -100, 640: -100},
    max_tokens= 2000,
    temperature= 0, # Optional, Defaults to 1. Range: 0 to 2
    top_p= 1 # Optional, Defaults to 1. It is generally recommended to alter this or temperature but not both.
    )

    response = chat_completion.choices[0].message.content

    queries = response.split('\n\n')
    
    retrievedChunks = []
    for sentence in queries:
        chunks = retriever.invoke(sentence)
        for chunk in chunks:
            retrievedChunks.append(chunk)
    sortedArray = []
    for doc in retrievedChunks:
        if doc not in sortedArray:
            sortedArray.append(doc)

    return sortedArray


In [29]:
multiQueryRetrieval("what are the various parts of a printer in a Deskjet3700")

[Document(metadata={'source': '..\\Data\\multiplePrinterBooks\\HP - Deskjet-3700-Printer.pdf', 'file_path': '..\\Data\\multiplePrinterBooks\\HP - Deskjet-3700-Printer.pdf', 'page': 0, 'total_pages': 118, 'format': 'PDF 1.6', 'title': 'HP DeskJet 3700 All-in-One series User Guide – ENWW', 'author': 'HP IWS Design Customer Education', 'subject': '', 'keywords': 'Edition 1, 9/2016', 'creator': 'AH XSL Formatter V6.2 MR6 for Windows (x64) : 6.2.8.19325 (2015/01/06 12:40JST)', 'producer': 'Antenna House PDF Output Library 6.2.640 (Windows (x64))', 'creationDate': "D:20160524165601+08'00'", 'modDate': "D:20160525102417+08'00'", 'trapped': ''}, page_content='HP DeskJet 3700 All-in-One series\n'),
 Document(metadata={'source': '..\\Data\\multiplePrinterBooks\\HP-Deskjet-4100-Troubeshooting.pdf', 'file_path': '..\\Data\\multiplePrinterBooks\\HP-Deskjet-4100-Troubeshooting.pdf', 'page': 1, 'total_pages': 19, 'format': 'PDF 1.7', 'title': '', 'author': 'Vardhan, Jai', 'subject': '', 'keywords': '